# Fetch Country

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import logging
import os
import re
import ast
from sklearn.preprocessing import MultiLabelBinarizer

In [7]:
recipes_df = pd.read_csv(os.path.join('/Users/mohammed_elamine/Documents/00 - AA - Learning/Telecom Paris - MS Data/1st Period/Kit Big Data/mangetamain/data/RAW_recipes.csv'))
recipes_df.head(3)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13


In [10]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mohammed_elamine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
recipes_df.loc[recipes_df['tags'] == "['']", 'tags'].head(3)

882     ['']
1257    ['']
1450    ['']
Name: tags, dtype: object

In [22]:
def clean_tag(tag):
    """Cleans the input tag by applying various preprocessing steps.

    Args:
        tag (str): The input tag to be cleaned.

    Returns:
        str: The cleaned tag or None if the tag is invalid.
    """
    
    if pd.isna(tag) or tag == '':
        return None  # mark for deletion
    
    # Remove leading and trailing spaces
    tag = tag.strip()

    # Lowercase everything
    tag = tag.lower()
    
    # Remove invalid/malformed tags
    if re.search(r"less_than|greater_than|sql", tag):
        return None  # mark for deletion
    
    # If it contains a space → treat as sentence
    if " " in tag:
        # Remove punctuation
        tag_clean = re.sub(r"[^\w\s]", "", tag)
        # Remove stopwords
        words = [w for w in tag_clean.split() if w not in stop_words]
        # Join with dash
        tag_clean = "-".join(words)
        return tag_clean
    
    # Otherwise, keep as is
    return tag

def extract_list_string(s):
    """Extracts a list of strings from a string representation of a list.

    Args:
        s (str): A string representation of a list.

    Raises:
        ValueError: If the input string does not represent a list.

    Returns:
        list: A list of strings extracted from the input string.
    """
    if pd.isna(s) or s == '':
        return []
    list_tags = ast.literal_eval(s)
    # Check if the result is indeed a list
    if isinstance(list_tags, list):
        # to only keep unique cleaned tags
        set_tags_clean = set()
        for i in range(len(list_tags)):
            cle_tag = clean_tag(list_tags[i])
            # Only add non-empty cleaned tags
            if cle_tag is not None:
                set_tags_clean.add(cle_tag)
        list_tags_clean = list(set_tags_clean)
        return list_tags_clean
    else:
        raise ValueError("The input string does not represent a list.")

recipes_df['tags_list'] = recipes_df['tags'].map(extract_list_string)
recipes_df[['tags', 'tags_list']].head(3)

,tags,tags_list
0,"['60-minutes-or-less', 'time-to-make', 'course...","[squash, occasion, vegetables, main-ingredient..."
1,"['30-minutes-or-less', 'time-to-make', 'course...","[breakfast, pizza, occasion, meat, equipment, ..."
2,"['time-to-make', 'course', 'preparation', 'mai...","[dietary, chili, time-to-make, course, prepara..."


In [23]:
mlb = MultiLabelBinarizer()
mlb.fit(recipes_df['tags_list'])

,classes,None
,sparse_output,False


In [24]:
mlb.classes_

array(['1-day-or-more', '15-minutes-or-less', '3-steps-or-less',
       '30-minutes-or-less', '4-hours-or-less', '5-ingredients-or-less',
       '60-minutes-or-less', 'a1-sauce', 'african', 'american',
       'amish-mennonite', 'angolan', 'appetizers', 'apples',
       'april-fools-day', 'argentine', 'artichoke', 'asian', 'asparagus',
       'australian', 'austrian', 'avocado', 'bacon', 'baja',
       'baked-beans', 'baking', 'bananas', 'bar-cookies', 'barbecue',
       'bass', 'bean-soup', 'beans', 'beans-side-dishes', 'bear', 'beef',
       'beef-barley-soup', 'beef-crock-pot', 'beef-kidney', 'beef-liver',
       'beef-organ-meats', 'beef-ribs', 'beef-sauces', 'beef-sausage',
       'beginner-cook', 'beijing', 'belgian', 'berries', 'beverages',
       'birthday', 'biscotti', 'bisques-cream-soups', 'black-bean-soup',
       'black-beans', 'blueberries', 'bok-choys', 'brazilian',
       'bread-machine', 'bread-pudding', 'breads', 'breakfast',
       'breakfast-casseroles', 'breakfast-e